In [21]:
import azureml.core

In [22]:
print(azureml.core.VERSION)

1.17.0


# Create Workspace

In [23]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

sid = '<subscription id>'
forced_interactive_auth = InteractiveLoginAuthentication(tenant_id="<tenant id>", force=True)

ws = Workspace.create(name='azureml_workspace2',
            subscription_id= sid, 
            resource_group='rgazureml',
            create_resource_group = True,
            location='centralus'
            )

WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


Deploying KeyVault with name azuremlwkeyvault1c5ecc36.
Deploying AppInsights with name azuremlwinsights72a1b356.
Deployed AppInsights with name azuremlwinsights72a1b356. Took 7.62 seconds.
Deploying StorageAccount with name azuremlwstorage995f102cf.
Deployed KeyVault with name azuremlwkeyvault1c5ecc36. Took 24.35 seconds.
Deployed StorageAccount with name azuremlwstorage995f102cf. Took 30.4 seconds.
Deploying Workspace with name azureml_workspace2.
Deployed Workspace with name azureml_workspace2. Took 35.68 seconds.


# Upload to Default Data Store

In [24]:
#upload data by using get_default_datastore()
ds = ws.get_default_datastore()
ds.upload(src_dir='./winedata', target_path='winedata', overwrite=True, show_progress=True)

print('Done')

Uploading an estimated of 2 files
Uploading ./winedata\winequality_red.csv
Uploaded ./winedata\winequality_red.csv, 1 files out of an estimated total of 2
Uploading ./winedata\wine.csv
Uploaded ./winedata\wine.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Done


# Create the Training Folder

In [25]:
import os

# create the folder
folder_training_script = './winecode'
os.makedirs(folder_training_script, exist_ok=True)

print('Done')

Done


# Create the Compute Cluster

In [26]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# Step 1: name the cluster and set the minimal and maximal number of nodes 
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 1)

# Step 2: choose environment variables 
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")

provisioning_config = AmlCompute.provisioning_configuration(
    vm_size = vm_size, min_nodes = min_nodes, max_nodes = max_nodes)

# create the cluster
compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

print('Compute target created')

Compute target created


# Create the Training Script

In [27]:
%%writefile $folder_training_script/train.py

import argparse
import os
import numpy as np
import pandas as pd
import glob

from azureml.core import Run
# from utils import load_data

import joblib

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

# let user feed in 2 parameters, the dataset to mount or download, and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--max-depth', type=float, dest='max_depth', default=5, help='max depth')
args = parser.parse_args()

###
data_folder = os.path.join(args.data_folder, 'winedata')
print('Data folder:', data_folder)

wine_data = pd.read_csv(os.path.join(data_folder, 'winequality_red.csv'))

                        
X = wine_data.drop(columns =["quality"])
y = wine_data["quality"]

clf = DecisionTreeRegressor(random_state=0,max_depth = args.max_depth)
rmse= np.mean(np.sqrt(-cross_val_score(clf, X, y, scoring="neg_mean_squared_error", cv = 5)))
print('RMSE is', rmse)

# Get the experiment run context
run = Run.get_context()

run.log('max depth', np.float(args.max_depth))
run.log('rmse', np.float(rmse))

os.makedirs('outputs', exist_ok=True)

clf.fit(X,y)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=clf, filename='outputs/wine_model.pkl')

run.complete()

Overwriting ./winecode/train.py


# Create the Environment

In [28]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
wine_env = Environment("wine-experiment-env")
wine_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
wine_env.docker.enabled = False # Use a docker container

# Create a set of package dependencies (conda or pip as required)
wine_packages = CondaDependencies.create(conda_packages=['scikit-learn'])

# Add the dependencies to the environment
wine_env.python.conda_dependencies = wine_packages

print(wine_env.name, 'defined.')

wine-experiment-env defined.


# Register the environment in the workspace

In [29]:
# Register the environment
wine_env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200821.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "wine-experiment-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda

# Create the Estimator

In [30]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.as_mount(),
    '--max-depth': 10
}

registered_env = Environment.get(ws, 'wine-experiment-env')

# Create an estimator
estimator = Estimator(source_directory=folder_training_script,
                      script_params=script_params,
                      compute_target = compute_target, # Run the experiment on the remote compute target
                      environment_definition = registered_env,
                      entry_script='train.py')

# Create the Experiment

In [31]:
from azureml.core import Experiment

#Create an experiment
experiment = Experiment(workspace = ws, name = "wine_expt")

print('Experiment created')

Experiment created


# Submit the Experiment with the Estimator information

In [32]:
run = experiment.submit(config=estimator)
run

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


Experiment,Id,Type,Status,Details Page,Docs Page
wine_expt,wine_expt_1615730273_039f33c8,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#get the result
print(run.get_metrics())

# Register the Model

In [ ]:
model = run.register_model(model_name='wine_model',
                           model_path='outputs/wine_model.pkl',
                           tags = {'area': "wine", 'type': "sklearn"},
                           description = "quality wine")

print(model.name, model.id, model.version, sep='\t')

# Inference Section

## Create the Inference Script

In [ ]:
%%writefile $folder_training_script/score.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('wine_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    log_txt = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_txt)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()

#  Create the Dependencies for the Inference Script

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = './winecode/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Create an Inference Configuration

In [ ]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = './winecode',
                                              entry_script="score.py",
                                              conda_file="env.yml")

# Create the Inference Cluster

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

# Deploy the Model to the Inference Cluster

In [ ]:
from azureml.core.model import Model

model = ws.models['wine_model']
service = Model.deploy(workspace=ws,
                       name = 'wine-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)

In [ ]:
service.update(enable_app_insights=True)

# Perform predictions with the Inference Cluster

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

In [ ]:
primary_key, secondary_key = service.get_keys()

In [ ]:
primary_key

In [ ]:
import requests
import json

# An array of new data cases
x_new = [[7.3,0.65,0,1.2,0.065,15,21,0.9946,3.39,0.47,10]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { "Content-Type":"application/json",
                    "Authorization":"Bearer " + primary_key }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

print(response)

In [ ]:
response.json()